In [1]:
import os

In [2]:
%pwd

'/workspaces/End-to-End-Gender-Classification-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/workspaces/End-to-End-Gender-Classification-project'

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    DATASET: Path
    male_path: Path
    female_path: Path
    data: Path
    image_folder1: Path
    folder_name: Path

In [14]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        config = self.config.data_preprocessing

        create_directories([config.root_dir])
        create_directories([config.DATASET])
        create_directories([config.male_path])
        create_directories([config.female_path])

        data_preprocessing_config = DataPreprocessingConfig(
            root_dir=config.root_dir,
            DATASET=config.DATASET,
            male_path=config.male_path,
            female_path=config.female_path,
            data=config.data,
            image_folder1=config.image_folder1,
            folder_name=config.folder_name 
        )

        return data_preprocessing_config
      

In [16]:
import pandas as pd
import os
from shutil import copyfile
import cv2
import numpy as np
from src.cnnClassifier import logger
import shutil


In [17]:
class DataPreprocessing:
    def __init__(self, config: DataPreprocessingConfig):
        self.config = config
 
    def get_celeba_images(self):
        data=pd.read_csv(self.config.data)
        # Select male and female images
        male_images = data[data['Male'] == 1]['image_id'].tolist()
        female_images = data[data['Male'] == -1]['image_id'].tolist()

        image_folder1=self.config.image_folder1

        # Copy male images
        for img in male_images[:1000]:  # Use first 1000 images
            img_path = os.path.join(image_folder1, img)  # Full path of the image
            img_path=cv2.imread(img_path)
            img_path=cv2.resize(img_path,(224,224))
            resized_image_rgb = cv2.cvtColor(img_path, cv2.COLOR_BGR2RGB)
            img_path=np.array(resized_image_rgb)
            save_path = os.path.join(self.config.male_path, f"um-{img}")  # Destination path
            cv2.imwrite(save_path, img_path)
        # Copy female images
        for img in female_images[:1000]:  # Use first 1000 images
            img_path = os.path.join(image_folder1, img)  # Full path of the image
            img_path=cv2.imread(img_path)
            img_path=cv2.resize(img_path,(224,224))
            resized_image_rgb = cv2.cvtColor(img_path, cv2.COLOR_BGR2RGB)
            img_path=np.array(resized_image_rgb)
            save_path = os.path.join(self.config.female_path, f"uf-{img}")  # Destination path
            cv2.imwrite(save_path, img_path)
        logger.info(f"CelebA Data-preprocessing Done")
    def get_knowing_person_images(self):
        folder_name=self.config.folder_name
        for sub_folder_name1 in os.listdir(folder_name):
            sub_folder_name=os.path.join(folder_name,sub_folder_name1)
            if os.path.isdir(sub_folder_name):
                images=os.listdir(sub_folder_name)
                for img_name1 in images:
                    img_name = os.path.join(sub_folder_name, img_name1)

                    # Load image
                    image = cv2.imread(img_name)

                    # Resize image (if necessary)
                    resized_image = cv2.resize(image, (224, 224))
                    resized_image_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
                    resized_image=np.array(resized_image_rgb)

                    #image=resized_image.convert('RGB')
                    '''
                    # Adjust brightness and contrast
                    adjusted_image = cv2.convertScaleAbs(resized_image, alpha=1.3, beta=40)  # alpha is contrast, beta is brightness

                    # Apply sharpening
                    kernel = np.array([[0, -1, 0],
                                        [-1, 5,-1],
                                        [0, -1, 0]])
                    sharpened_image = cv2.filter2D(adjusted_image, -1, kernel)

                    # Denoise the image
                    denoised_image = cv2.fastNlMeansDenoisingColored(sharpened_image, None, 10, 10, 7, 21)
                    '''
                    # Save the enhanced image

                    save_path = os.path.join(sub_folder_name, f'enhance-{img_name1}')
                    cv2.imwrite(save_path,resized_image)
                    os.remove(img_name)
        
    def knowing_person_images2(self):
        main_folder = self.config.folder_name
        for sub_dirs in os.listdir(main_folder):
            sub_dir_path = os.path.join(main_folder, sub_dirs)
            dest_dir_path = os.path.join(self.config.DATASET, sub_dirs)  # Define the destination path

            # Check if the destination folder exists
            if os.path.exists(dest_dir_path):
                shutil.rmtree(dest_dir_path)  # Remove the existing destination folder
                logger.info(f"Removed existing folder '{dest_dir_path}'")

            shutil.move(sub_dir_path, self.config.DATASET)  # Move the source folder to destination
            logger.info(f"Moved '{sub_dir_path}' to '{self.config.DATASET}'")

        logger.info(f"Knowing Person Data-preprocessing Done")




        

In [18]:
try:
    config = ConfigurationManager()
    data_preprocessing_config = config.get_data_preprocessing_config()
    data_peprocessing = DataPreprocessing(config=data_preprocessing_config)
    data_peprocessing.get_celeba_images()
    data_peprocessing.get_knowing_person_images()
    data_peprocessing.knowing_person_images2()
except Exception as e:
    raise e

[2025-01-07 01:13:19,191: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-07 01:13:19,721: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-07 01:13:19,722: INFO: common: created directory at: artifacts]
[2025-01-07 01:13:19,723: INFO: common: created directory at: artifacts/data_preprocessing]
[2025-01-07 01:13:20,388: INFO: common: created directory at: artifacts/data_preprocessing/DATASET]
[2025-01-07 01:13:20,415: INFO: common: created directory at: artifacts/data_preprocessing/DATASET/male]
[2025-01-07 01:13:20,416: INFO: common: created directory at: artifacts/data_preprocessing/DATASET/female]
[2025-01-07 01:13:38,595: INFO: 625737029: CelebA Data-preprocessing Done]
[2025-01-07 01:13:49,869: INFO: 625737029: Moved 'artifacts/data_ingestion/knowing-person-images/M(Farhan)' to 'artifacts/data_preprocessing/DATASET']
[2025-01-07 01:13:49,877: INFO: 625737029: Moved 'artifacts/data_ingestion/knowing-person-images/M(Fahad)' to 'artifacts/d